# Sequence Parameters

## *Sequence Type*: predefined sequence or waveform upload?

Many use cases require the freedom to define waveforms on a sample-basis. The `"Simple"` sequence type provided by the `zhinst-toolkit` allows for exactly that. If the *Simple* sequence is configured, the user can add waveforms to a queue of waveforms to be uploaded to the AWG Core. All the queued up waveforms will then be played in that order. The waveforms are defiend as simple `numpy` arrays, with every value in the array corresponding to one sample. Since the waveforms are defined purely by samples, the duration of the waveform depends on the *sampling rate* of the AWG Core.

Instead of uploading waveforms defined as *numpy* arrays, the `zhinst-toolkit` also provides pre-defined sequences for standard experiments. These differ between UHFQA and HDAWG and include:
 
 * Rabi (HDAWG)
 * T1 (HDAWG)
 * T2 Ramsey (HDAWG)
 * Trigger (HDAWG)
 * Continuous-Wave (CW) Spectroscopy (UHFQA)
 * Pulsed Spectroscopy (UHFQA)
 * Multiplexed Readout (UHFQA)

## Definition of a sequence: *period* and time origin

The parameter `period` defines the duration in seconds of one single shot of the experiment. The start of each period in the sequence is defined by a trigger signal (either the AWG sends or receives a trigger). The time origin `t=0` is the reference point for all waveform. It is defined to not coincide with the trigger signal but to be a well defined time after the trigger. This way, it is possible to shift waveforms to  *negative time* with respect to the orogin `t=0`.

The parameter `dead_time` describes the the time in seconds after the time origin `t=0` before the next period begins, i.e. before the next trigger signal is sent or expected. The time origin is thus at `period - dead_time` after the trigger signal. The *dead time* is important to keep in mind because it defines the maximum length of a waveform that can be played *after* `t=0`. If the waveform is longer than *dead time*, the next trigger signal will be missed. The *dead time* defaults to *5 us*.

```

Trigger Signal                                                 Time Origin t=0
+                                                              +                                +
|                                                              |                                |
|                                                              |                                |
|                                                              |                                |
+--------------------------------------------------------------+--------------------------------+
|                                                              |                                |
|                                                              |                                |
| <------------------------------------+  Period  +------------------------------------------>  |
|                                                              |                                |
|                                                              | <--------+ Dead Time +------>  |
+                                                              +                                +

```

## *Repetitions*

The parameter `repetitions` defines the number of repetitions in the outer loop of the experiment. For example in an experiment with a single waveform (e.g. a single queued waveform or a Rabi or T1 experiment with a single point) the waveform will be repeated `repetitions` times for a total of `1 x repetitions` shots.


```
+---+                                        +---+
|                                                |
|    +                     XXXX             +    |
|    |                    XXXXXX            |    |
|    +------------------XXXXXXXXXX----------+    |
|    |                                      |    |    X  ( Repetitions )
|    |   <---------+  Period  +--------->   |    |
|    +                                      +    |
|                                                |
+---+                                        +---+
```

The corresponding pulse sequence would look like this:

```
+              XXXX        +            XXXX         +          XXXX         +             XXXX         +
|             XXXXXX       |           XXXXXX        |         XXXXXX        |            XXXXXX        |
+-----------XXXXXXXXXX-----+---------XXXXXXXXXX------+-------XXXXXXXXXX------+----------XXXXXXXXXX------+  -  -  -
|                          |                         |                       |                          |
+                          +                         +                       +                          +

```

Note however that for an experiment with multiple points in the inner loop (e.g. multiple waveforms in the queue or a Rabi or T1 experiment with multiple points) the parameter `repetitions` only refers to the outer loop. For example a Rabi experiment with `N` different amplitudes

```
+---+                                                                           +---+
|                                                                     XX            |
|    +                        +               XXX      +             XXXX      +    |
|    |              XXXX      |              XXXXX     |            XXXXXX     |    |
|    +-------------XXXXXX-----+-------------XXXXXXX----+-----------XXXXXXXX ---+    |
|    |                        |                        |                       |    |    X  ( Repetitions )
|    |       Amp 1            |      Amp ...           |      Amp N            |    |
|    +                        +                        +                       +    |
|                                                                                   |
+---+                                                                           +---+
```
would mean `N x repetitions` shots in total.

## Waveform *alignment*

The waveforms within each period are always played relative to the time origin at `t=0`. Often, it is desireable to start one waveform right after another waveform has ended. In order to allow gapless playback on different AWGs, the parameter `alignment` defines if the waveform should end at the time origin or start with the time origin.

* `alignment = "End with Trigger"`: the waveform is played *before* the time origin `t=0` and ends at `t=0`

```
                                                               Time Origin t=0
                                                               +
                                           Waveform            |
+                                          XXXXXXXXXXXXXXXXXXX |                                +
|                                         XXXXXXXXXXXXXXXXXXXXX|                                |
|        Alignment: "End with Trigger"   XXXXXXXXXXXXXXXXXXXXXXX                                |
|                                        XXXXXXXXXXXXXXXXXXXXXXX                                |
+----------------------------------------XXXXXXXXXXXXXXXXXXXXXXX--------------------------------+
|                                                                                               |
|                                                                                               |
|                                                                                               |
+                                                                                               +

```

* `alignment = "Start with Trigger"`: the waveform is played *after* the time origin `t=0` and starts at `t=0`

```

                                                               Time Origin t=0
                                                               +
                                                               | Waveform
+                                                              | XXXXXXXXXXXXXXXXXXXXXX         +
|                                                              |XXXXXXXXXXXXXXXXXXXXXXXX        |
|        Alignment: "Start with Trigger"                       XXXXXXXXXXXXXXXXXXXXXXXXXX       |
|                                                              XXXXXXXXXXXXXXXXXXXXXXXXXX       |
+--------------------------------------------------------------XXXXXXXXXXXXXXXXXXXXXXXXXX-------+
|                                                                                               |
|                                                                                               |
|                                                                                               |
+                                                                                               +

```

The parameter `alignment` can also be used with the Enum `Alignment` available in the `zhinst-toolkit`:
```python
from zhinst.toolkit import Alignment

hdawg.awgs[0].set_sequence_params(alignment="Start with Trigger")
hdawg.awgs[0].set_sequence_params(alignment=Alignment.START_WITH_TRIGGER)
```

## Using multiple AWGs: *trigger mode*

The correct trigger configuration is key to enable the aligned playback of waveforms over multiple AWG Cores and instruments. The standard configuration within the `zhinst-toolkit` assumes a **Master Trigger** configuration with one AWG Core that sends out the trigger signal to all other AWG Cores in the experiment, including on different instruments like another HDAWG or a UHFQA.

```

               HDAWG 1
              +-------------+
+-------<-----+   AWG 1     |  Master Trigger
|             +-------------+
+-------+------>  AWG 2     |
        |     |-------------+
        +------>  AWG 3     |
        |     |-------------+
        +------>  AWG 4     |
        |     +-------------+
        |
        |      HDAWG 2
        |     +-------------+
        +------>  AWG 1     |
        |     |-------------+
        +------>  AWG 2     |
        |     |-------------+
        +------>  AWG 3     |
        |     |-------------+
        +------>  AWG 4     |
        |     +-------------+
        |
        |      UHFQA
        |     +-------------+
        +------>            |
              +-------------+


```

On the HDAWG, the trigger output of a AWG Core is labelled with *'Mark'* , the trigger input is labelled *'Trig'*. By convention, every AWG Core uses the *Mark* and *Trig* of the lower channel. On the UHFQA the *Ref/Trig 1* us used for both sending and receiving triggers.

```
       Out 1 ^    ^ Out 2                      Out 3 ^    ^ Out 4
             |    |                                  |    |
             |    |                                  |    |
AWG 1        |    |                     AWG 2        |    |
+------------+----+------------+        +------------+----+------------+
|            O    O            |        |            O    O            |
|  O    O              O    O  |        |  O    O              O    O  |
+-------+----------------------+        +--+---------------------------+
        |                                  ^
        | Mark 1                           | Trig In 3
        v                                  |
        +----->------>------->------->-----+

```

The parameter `trigger_mode` specifies whether the AWG Core is used as a master trigger, expects a trigger or is used by itself.

* `trigger_mode = "Send Trigger"`: send a trigger signal at the start of each period to act as a *master trigger* AWG Core
* `trigger_mode = "External Trigger"`: wait for an external trigger signal at the start of every period, the AWG Core is triggered by another AWG Core


```
                                                                              Time Origin t=0
                                                              Waveform        +
                                                                              |
AWG 1                  +                                           XXXXXX     |                     +
                       |                                         XXXXXXXXXX   |                     |
Trigger Mode:          |                                        XXXXXXXXXXXX  |  Alignment:         |
"Send Trigger"         |                                       XXXXXXXXXXXXXX |  "End with Trigger" |
                       +-------------------------------------XXXXXXXXXXXXXXXXXX---------------------+
                       |                                                      |                     |
                       |                                                      |                     |
                       |   Send Trigger Signal                                |                     |
                       |      (Mark 1)                                        |                     |
                     \ | /                                                    |                     |
                      \|/                                                     |                     |
                       v                                                      |                     |
                                                                              |                     |
AWG 2                  +                                                      |  Waveform           |
                       |                                                      |                     |
Trigger Mode:          |   Wait for Trigger Signal                            |     XXXXXX          |
"External Trigger"     |      (Trigger 3)                                     |   XXXXXXXXXX        |
                     \ | /                               Alignment:           |  XXXXXXXXXXXX       |
                      \|/                                "Start with Trigger" | XXXXXXXXXXXXXX      |
                       v------------------------------------------------------XXXXXXXXXXXXXXXXXX----+

```
If the *trigger mode* of all AWG Cores is configured correctly, their *time origins* coincide and the waveforms on all AWGs can be aligned with respect to `t=0`. Note that other sequence parameters like `period`, `dead_time` , `repetitions` have to be identical for the different AWG Cores in order to work together as expected!

* `trigger_mode = "None"`: to not send out a trigger signal at the start of every period, this is the default setting and can be used when the AWG Core does not have to trigger any other AWGs

The parameter `trigger_mode` can also be used with the Enum `TriggerMode` available in the `zhinst-toolkit`:
```python
from zhinst.toolkit import TriggerMode

hdawg.awgs[0].set_sequence_params(trigger_mode="Send Trigger")
hdawg.awgs[0].set_sequence_params(trigger_mode=TriggerMode.SEND_TRIGGER)
```

### *Trigger delay*

An additional parameter `trigger_delay` can be used to shift the reference point for a given AWG Core. The parameter acts as a delay in the trigger signal. All waveforms in the sequence will be shifted by *trigger_delay* with respect to the time origin `t=0`.

```
                                      Time Origin t=0
    Trigger Mode:                     +
 |  "External Trigger"                |          
 |                                    |        Waveform
 |  Alignment:                        |              XXXX
\|/ "Start with Trigger"              |            XXXXXXXX        +
 v                                    |           XXXXXXXXXX       |
 +------------------------------------+----------XXXXXXXXXXXX------+
 |                                    |          |                 |
 +                                    + <------> +                 +
                                       Trigger Delay


                                      Time Origin t=0
    Trigger Mode:                     +
 |  "External Trigger"                |
 |                                Waveform
 |  Alignment:                        | XXXX 
\|/ "End with Trigger"                XXXXXXXX                     +
 v                                   XXXXXXXXXX                    |
 +----------------------------------XXXXXXXXXXXX-------------------+
 |                                    |        |                   |
 +                                    +<------>+                   +
                                       Trigger Delay


```

Have a look at the examples provided for more information on how to trigger AWG Cores and how to program typical experiments with the HDAWG and UHFQA.